In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# load data
df = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv')

In [3]:
# overlook of data
df.head(n=5)

In [4]:
# columns
df.columns

In [18]:
df['Hillshade'] = df['Hillshade_9am'] + df['Hillshade_Noon'] + df['Hillshade_3pm']

In [19]:
def plot_features(features):
    for feature in features:
        plt.rcParams["figure.figsize"] = (20,7)
        for ctype in list(df['Cover_Type'].unique()):
            values = df.loc[df['Cover_Type'] == ctype][feature].values
            plt.scatter(x=values, y=np.arange(values.size), label=f"Cover type {ctype}")
        plt.title(feature)
        plt.legend()
        plt.show()

In [22]:
# Scatter plots for features with contnous values
plot_features(['Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade', 'Horizontal_Distance_To_Fire_Points'])